# Module Import

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
#import tqdm.notebook.tqdm as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

from PIL import Image
from pathlib import Path
from tqdm.notebook import trange, tqdm

import csv
import pandas as pd

# データセット用CSV作成

In [ ]:
# img_dir = 'datasets/train'
# param = np.load('datasets/train/param.npy')
# i = 0

# filename = 'datasets/train.csv'
# # ヘッダー作成
# with open(filename, 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(['img','param'])

# # ファイル名書き込み
# for subdir, dirs, files in os.walk(img_dir):
#     for file in sorted(files):
#         file_path = os.path.join(subdir, file)
#         if file_path.endswith(".png"):
#             with open(filename, 'a', newline="") as f:
#                 writer = csv.writer(f)
#                 writer.writerow([file_path, param[i]])
#                 i += 1

In [ ]:
# img_dir = 'datasets/test'
# param = np.load('datasets/test/param.npy')
# i = 0

# filename = 'datasets/test.csv'
# # ヘッダー作成
# with open(filename, 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(['img','param'])

# # ファイル名書き込み
# for subdir, dirs, files in os.walk(img_dir):
#     for file in sorted(files):
#         file_path = os.path.join(subdir, file)
#         if file_path.endswith(".png"):
#             with open(filename, 'a', newline="") as f:
#                 writer = csv.writer(f)
#                 writer.writerow([file_path, param[i]])
#                 i += 1

# My dataset

In [2]:
class ImageDataset(Dataset):
    def __init__(self, csv_file, np_file, transform=None):
        df = pd.read_csv(csv_file)
        img_paths = df['img']
        labels = np.load(np_file)
        
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = Image.open(img_path)

        if self.transform is not None:
            img = self.transform(img)
            
        label = torch.from_numpy(self.labels[index]).float()

        return img, label
    
    def __len__(self):
        return len(self.img_paths)

In [3]:
# Transform を作成する。
transform = torchvision.transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Dataset を作成する。
dataset = ImageDataset(csv_file='datasets/train.csv', np_file='datasets/train/param.npy', transform=transform)
train_size = int(len(dataset) * 0.8)
validation_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, validation_size])
test_dataset = ImageDataset(csv_file='datasets/test.csv', np_file='datasets/test/param.npy', transform=transform)

In [4]:
print(dataset)
print(train_dataset, train_size)
print(val_dataset,validation_size)
print(test_dataset[0][0].size(), test_dataset[0][1])

<torch.utils.data.dataset.Subset object at 0x7f47edafa9d0> 40000
<torch.utils.data.dataset.Subset object at 0x7f47ed8c8bd0> 10000
torch.Size([3, 256, 256]) tensor([-0.5781, -4.4716,  4.3201, -1.1764,  1.9480,  0.1977,  0.1513, -0.3541,
         0.2071,  1.5665,  0.4418,  0.1889,  0.8275,  0.2787, -0.5138, -0.3628,
         0.5325, -0.3812, -0.1636, -0.0221, -0.2208, -0.3374, -0.2170,  0.0353,
        -0.7814,  0.1090,  1.0405,  0.5513, -0.4480, -0.6760,  0.3750, -0.6260,
        -0.0900,  0.0905,  0.8560,  0.6698, -0.9916, -0.4202, -0.5007,  0.0403,
         0.0739,  0.7468, -0.1637,  0.4360,  0.3696, -0.6656, -0.4994,  0.2244,
        -0.2650, -0.8427,  0.7259, -0.5026, -0.4785,  0.3518, -0.3648, -0.3544,
        -0.1964,  0.2229, -0.5456, -0.2831, -0.3923,  0.4116, -0.2047,  0.1248,
         0.0537,  0.2337,  0.4770,  0.4006, -0.2927, -0.0558,  0.4819, -0.4151,
         0.0436,  0.2702,  0.1255,  0.1450, -0.0529,  0.0692, -0.4968, -9.1243])


In [ ]:
loader = DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=4, drop_last=False)
sample, _ = next(iter(loader))
grid = torchvision.utils.make_grid(sample, nrow=8, padding=1, normalize=True)
plt.rcParams["figure.figsize"] = (64, 48)
plt.imshow(grid.numpy().transpose(1,2,0))

In [ ]:
loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, drop_last=False)
sample, _ = next(iter(loader))
grid = torchvision.utils.make_grid(sample, nrow=8, padding=1, normalize=True)
plt.rcParams["figure.figsize"] = (64, 48)
plt.imshow(grid.numpy().transpose(1,2,0))

# Model Building

In [5]:
class EncoderBlock(nn.Module):
  def __init__(self, in_feature, out_feature):
    super(EncoderBlock, self).__init__()
    self.in_feature = in_feature
    self.out_feature = out_feature

    layers = []
    layers.append(nn.Conv2d(in_channels=in_feature, out_channels=out_feature, kernel_size=3, stride=1, padding=1))
    layers.append(nn.BatchNorm2d(out_feature))
    layers.append(nn.LeakyReLU(negative_slope=0.01, inplace=True))
    self.block = nn.Sequential(*layers)

  def forward(self, x):
    return self.block(x)

In [6]:
class Encoder(nn.Module):
  def __init__(self, dim):
    super(Encoder, self).__init__()

    self.conv1 = EncoderBlock(3  , dim)
    self.conv2 = EncoderBlock(dim, dim)
    self.conv3 = EncoderBlock(dim, dim)
    self.conv4 = EncoderBlock(dim, dim*2)
    self.conv5 = EncoderBlock(dim*2, dim*2)
    self.conv6 = EncoderBlock(dim*2, dim*2)
    self.conv7 = EncoderBlock(dim*2, dim*4)
    self.conv8 = EncoderBlock(dim*4, dim*4)
    self.conv9 = EncoderBlock(dim*4, dim*4)
    self.pool = nn.MaxPool2d(kernel_size=2)

  def forward(self, x):
    out = self.conv1(x)
    out = self.conv2(out)
    out = self.conv3(out)
    out = self.pool(out) 

    out = self.conv4(out)
    out = self.conv5(out)
    out = self.conv6(out)
    out = self.pool(out)
    
    out = self.conv7(out)
    out = self.conv8(out)
    out = self.conv9(out)
    out = self.pool(out)

    return out

In [7]:
class Classifier(nn.Module):
  def __init__(self, class_num, enc_dim, in_w, in_h):
    super(Classifier, self).__init__()

    self.enc_dim = enc_dim
    self.in_w = in_w
    self.in_h = in_h
    self.fc_dim = enc_dim*4 * int(in_h/2/2/2) * int(in_w/2/2/2)

    self.Encoder = Encoder(self.enc_dim)
    self.fc1 = nn.Linear(self.fc_dim, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 80)
  
  def forward(self, x):
    out = self.Encoder(x)
    out = out.view(-1, self.fc_dim)

    out = F.relu(self.fc1(out))
    out = F.relu(self.fc2(out))
    out = self.fc3(out)

    return out


# Train

In [8]:
num_epochs = 100

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
net = Classifier(class_num=80, enc_dim=128, in_w=256, in_h=256)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

cuda
Let's use 2 GPUs!


In [10]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, drop_last=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, drop_last=False)

In [11]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    @property
    def avg(self):
        return self.sum / self.count

    def reset(self):
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n

class History(object):

    def __init__(self, keys, output_dir):
        #self.output_dir = output_dir
        self.keys = keys

        self.logs = {key: [] for key in keys}

    def __call__(self, data):
        for key, value in data.items():
            self.logs[key].append(value)

    def save(self, filename='history.pkl'):
        savepath = os.path.join(self.output_dir, filename)
        with open(savepath, 'wb') as f:
            pickle.dump(self.logs, f)

    def plot_graph(self, filename='loss.png'):
        fig, ax = plt.subplots()
        ax.set_xlabel('Iteration')
        ax.set_ylabel('Loss')
        ax.set_title('Training curve.')
        ax.set_ylim(0, 2)

        for i, (key, value) in enumerate(self.logs.items()):
            x = np.arange(len(value))
            y = np.array(value)
            ax.plot(x, y, label=key, color=cm.cividis(i / len(self.logs.keys())))
        
        ax.legend(loc='best')

        plt.show()
        #save_path = os.path.join(self.output_dir, filename)
        #logger.info('Save {}'.format(save_path))
        #plt.savefig(save_path, transparent=True)
        plt.clf()
        plt.cla()
        plt.close('all')

In [ ]:
# Training. {{{
# =====
train_history = History(keys=('train_loss', '_t'), output_dir=None)
val_history = History(keys=('val_loss', '_t'), output_dir=None)

for epoch in range(num_epochs):
  loop = tqdm(train_loader, unit='batch', desc='Epoch {:>3}'.format(epoch+1))
  correct = 0
  total = 0

  # Train Step. {{{
  # =====
  net.train()
  for i, batch in enumerate(loop):
    G_meter = AverageMeter()
    inputs, labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Update network. {{{
    # =====
    # forward network
    outputs = net(inputs)

    # backward network
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    # }}}

    # Get losses. {{{
    G_meter.update(loss.item(), inputs[0].size()[0])
    train_history({'train_loss': loss.item()})
    # }}}

    # Calcurate Accuracy. {{{
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
#     correct += (predicted == labels).sum().item()
    # }}}
  # }}}
  
  # Print training log. {{{
  # =====
  msg = "[Train {}] Epoch {}/{}".format(
      'CNN', epoch + 1, num_epochs)
  msg += " - {}: {:.3f}".format('train_loss', G_meter.avg)
#   msg += " - {}: {:.3f}".format('train_accuracy', correct / total)
  print(msg)
  # }}}

  # Validation Step. {{{
  # =====
  with torch.no_grad():
    net.eval()
    loop_val = tqdm(val_loader, unit='batch', desc='Epoch {:>3}'.format(epoch + 1))
    correct = 0
    total = 0

    for i, batch in enumerate(loop_val):
      G_meter = AverageMeter()
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = net(inputs)

      loss = criterion(outputs, labels)

      G_meter.update(loss.item(), inputs[0].size()[0])
      val_history({'val_loss': loss.item()})

      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
#       correct += (predicted == labels).sum().item()
  # }}}

  # Print validation log. {{{
  # =====
  msg = "[Validation {}] Epoch {}/{}".format(
      'CNN', epoch + 1, num_epochs)
  msg += " - {}: {:.3f}".format('val_loss', G_meter.avg)
#   msg += " - {}: {:.3f}".format('val_accuracy', correct / total)
  print(msg)
  # }}}

model_path = './prototype.pth'
torch.save(net.state_dict(), model_path)

Epoch   1:   0%|          | 0/625 [00:00<?, ?batch/s]

[Train CNN] Epoch 1/100 - train_loss: 0.833


Epoch   1:   0%|          | 0/157 [00:00<?, ?batch/s]

[Validation CNN] Epoch 1/100 - val_loss: 0.745


Epoch   2:   0%|          | 0/625 [00:00<?, ?batch/s]

...

Epoch  52:   0%|          | 0/625 [00:00<?, ?batch/s]

[Train CNN] Epoch 52/100 - train_loss: 1.002


Epoch  52:   0%|          | 0/157 [00:00<?, ?batch/s]

[Validation CNN] Epoch 52/100 - val_loss: 0.745


Epoch  53:   0%|          | 0/625 [00:00<?, ?batch/s]

In [ ]:
import matplotlib.cm as cm
plt.rcParams["figure.figsize"] = (6.4, 4.8)
train_history.plot_graph()
val_history.plot_graph()